###### 2020-11-23 월요일

# 02_Bidirect LSTM
 - 구글 collab으로 실행하였습니다

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import pandas as pd
import numpy as np
import re
import json
import time

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import accuracy_score, log_loss, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split

## 1. 데이터불러오기 & 합치기

In [4]:
X_train = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/X_train.csv', index_col=[0])
X_test = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/X_test.csv', index_col=[0])
y_train = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/y_train.csv', index_col=[0])
y_test = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/y_test.csv', index_col=[0])

word_index_json = open('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/word_index_vocab.json', 'r').read()
word_index_vocab = json.loads(word_index_json)

## 2. 데이터토큰화 & 패딩

#### 텍스트 음절단위로 자르기

In [5]:
X_train_split = X_train['댓글'].apply(list).tolist()
X_test_split = X_test['댓글'].apply(list).tolist()

#### 정수인덱싱

In [6]:
tokenizer = Tokenizer()
tokenizer.word_index = word_index_vocab

X_train_sequences = tokenizer.texts_to_sequences(X_train_split)
X_test_sequences = tokenizer.texts_to_sequences(X_test_split)

### 패딩

In [7]:
train = pad_sequences(X_train_sequences, padding='post', maxlen=400)
test = pad_sequences(X_test_sequences, padding='post', maxlen=400)

## 3. SMOTE를 이용한 라벨 불균형 해소

In [8]:
y_train.value_counts()

악플여부
0       54548
1       21408
dtype: int64

 - 라벨 0은 446162, 1은 2897, 2는 4993으로 균형이 잡혀있지않아 recall(재현율)이 떨어질 가능성이 높다
 - 그래서 SMOTE를 이용하여 라벨이 1과 2인 경우를 복제하여 라벨학습의 균형을 맞추어 주는 것이다
 - 이것을 `오버샘플링`이라 한다.

In [9]:
from imblearn.over_sampling import SMOTE

In [10]:
start = time.time()

smote = SMOTE(random_state=123)
train_over, y_train_over = smote.fit_sample(train, y_train)

print("걸린시간 :", time.time() - start)

걸린시간 : 226.50212502479553


In [11]:
pd.Series(y_train_over).value_counts()

1    54548
0    54548
dtype: int64

## 4. Bidirect LSTM 구현

In [12]:
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import f1_score

In [21]:
# 음절 단어사전의 크기
vocab_size = len(word_index_vocab)
embedding_dim = 16
comment_len = 400

In [22]:
model = Sequential()
model.add(Embedding(vocab_size + 1, embedding_dim, input_length=comment_len))
model.add(Bidirectional(LSTM(16, return_sequences=True)))
model.add(Bidirectional(LSTM(32, return_sequences=False)))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

In [15]:
# 성능의 변화가 없을때 멈추는 기능
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

# 지금까지 가장 좋은 성능이 나왔을때, 노드의 가중치를 저장하는 함수
mc = ModelCheckpoint('erase_label_1.h5', monitor= 'val_loss', mode='min', save_best_only=True)

In [16]:
y_train_over = y_train_over.reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

In [17]:
print(train_over.shape)
print(test.shape)
print(y_train_over.shape)
print(y_test.shape)



(109096, 400)
(18990, 400)
(109096, 1)
(18990, 1)


In [18]:
print(type(train_over))
print(type(test))
print(type(y_train_over))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [24]:
history = model.fit(train_over, y_train_over, 
                    callbacks        = [es, mc],
                    epochs           = 20,
                    batch_size       = 64, 
                    validation_data = (test, y_test))

Epoch 1/20
1705/1705 [==============================] - 106s 62ms/step - loss: 0.3741 - accuracy: 0.8346 - val_loss: 0.2263 - val_accuracy: 0.8978
Epoch 2/20
1705/1705 [==============================] - 105s 62ms/step - loss: 0.2861 - accuracy: 0.8819 - val_loss: 0.1440 - val_accuracy: 0.9410
Epoch 3/20
1705/1705 [==============================] - 105s 61ms/step - loss: 0.2536 - accuracy: 0.8961 - val_loss: 0.1344 - val_accuracy: 0.9428
Epoch 4/20
1705/1705 [==============================] - 104s 61ms/step - loss: 0.2712 - accuracy: 0.8872 - val_loss: 0.1408 - val_accuracy: 0.9350
Epoch 5/20
1705/1705 [==============================] - 104s 61ms/step - loss: 0.2367 - accuracy: 0.9038 - val_loss: 0.1650 - val_accuracy: 0.9265
Epoch 6/20
1705/1705 [==============================] - 105s 62ms/step - loss: 0.2261 - accuracy: 0.9086 - val_loss: 0.1316 - val_accuracy: 0.9445
Epoch 7/20
1705/1705 [==============================] - 104s 61ms/step - loss: 0.2113 - accuracy: 0.9155 - val_loss: 0

## 5. Test Set 예측

In [25]:
# 저장한 LSTM 모델을 불러오고 예측해보자

loaded_model = load_model('erase_label_1.h5')
y_pred = loaded_model.predict(test)

y_class = y_pred > 0.5
confusion_matrix(y_class, y_test)

array([[12114,   937],
       [ 1579,  4360]])

In [26]:
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score

print('recall : ', recall_score(y_class, y_test))
print('precision : ', precision_score(y_class, y_test))
print('f1_score : ', f1_score(y_class, y_test))
print('roc_auc_score : ', roc_auc_score(y_test, y_pred))

recall :  0.7341303249705338
precision :  0.8231074192939399
f1_score :  0.7760768956924172
roc_auc_score :  0.9295629431391224


## 5. 모델 테스트

In [27]:
def model_test(comment):
    comment_list = [list(comment)]
    comment_label = tokenizer.texts_to_sequences(comment_list)

    comment_pad = pad_sequences(comment_label, padding='post', maxlen=400)

    pred = loaded_model.predict(comment_pad)

    if pred < 0.5 :
        result = '악플이 아닙니다'
    else :
        result = '악플입니다'
    
    print(result)
    print('라벨확률 : ', pred)
    
    return result, pred

In [34]:
comment = ''''''

result, prob = model_test(comment)

악플이 아닙니다
라벨확률 :  [[0.4852688]]
